<a href="https://colab.research.google.com/github/zeeba-tech/data-science/blob/main/MLtitanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [3]:
print(df.shape)
print(df.size)

(891, 12)
10692


In [4]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:

print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
       PassengerId    Survived      Pclass  ...       SibSp       Parch        Fare
count   891.000000  891.000000  891.000000  ...  891.000000  891.000000  891.000000
mean    446.000000    0.383838    2.308642  ...    0.523008    0.381594

In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
len([x for x in df.Name])

891

In [8]:

def get_title(name):
  if '.' in name:
    return name.split(',')[1].split('.')[0].strip()
  else:
    return 'unknown'

# A list with the all the different titles
titles=sorted(set([x for x in df.Name.map(lambda x:get_title(x))]))
print('different titles found on the dataset:')
print(len(titles),':',titles)
print()

different titles found on the dataset:
17 : ['Capt', 'Col', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Master', 'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir', 'the Countess']



In [9]:
# normalise the titles

def replace_titles(x):
  title=x['Title']
  if title in ['Capt','Col','Major']:
    return 'Officer'
  elif title in ["Jonkheer","Don",'the countess','Dona','Lady',"Sir"]:
    return 'Royalty'
  elif title in ['the Countess','Mlle','Mme','Ms','Lady']:
    return 'Miss'
  else:
    return title

In [10]:




df['Title']=df['Name'].map(lambda x:get_title(x))

df['Title']=df.apply(replace_titles,axis=1)
print(df.Title.value_counts())

Mr         517
Miss       187
Mrs        125
Master      40
Dr           7
Rev          6
Officer      5
Royalty      4
Name: Title, dtype: int64


In [11]:
df['Age'].fillna(df['Age'].median(),inplace=True)
df['Fare'].fillna(df['Fare'].median(),inplace=True)
df['Embarked'].fillna("S",inplace=True)
df.drop("Cabin",axis=1 ,inplace=True)
df.drop("Name",axis=1,inplace=True)
df.drop('Ticket',axis=1,inplace=True)
df.Sex.replace(('male','female'),(0,1),inplace=True)
df.Embarked.replace(('S','C','Q'),(0,1,2),inplace=True)
df.Title.replace(('Mr','Miss','Mrs','Master','Dr','Rev','Officer','Royalty'),(0,1,2,3,4,5,6,7),inplace=True)


print(df.isnull().sum())
print(df['Sex'].sample(5))
print(df['Embarked'].sample(5))
print(df.columns)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Title          0
dtype: int64
99     0
158    0
423    1
213    0
560    0
Name: Sex, dtype: int64
77     0
218    1
701    0
705    0
515    0
Name: Embarked, dtype: int64
Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked', 'Title'],
      dtype='object')


In [12]:
corr=df.corr()
corr.Survived.sort_values(ascending=False)

Survived       1.000000
Sex            0.543351
Title          0.358824
Fare           0.257307
Embarked       0.106811
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.064910
Pclass        -0.338481
Name: Survived, dtype: float64

In [13]:
from sklearn.model_selection import train_test_split

x=df.drop(['Survived','PassengerId'],axis=1)
y=df['Survived']
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.1)

In [22]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

randomforest =RandomForestClassifier()
randomforest.fit(x_train,y_train)
y_pred=randomforest.predict(x_val)
acc_randomforest=round(accuracy_score(y_pred,y_val)*100,2)
print(acc_randomforest)

pickle.dump(randomforest,open('titanic_model.sav','wb'))

83.33
